
# London Underground Upgrade Impact Analysis

This notebook reproduces the analysis of London Underground upgrades between 2010 and 2024. It loads pre‑processed datasets of network performance metrics, ridership and reliability, integrates the upgrade timeline for each line, and generates visualisations such as the Excess Journey Time (EJT) trend by line, an EJT heatmap, and a Lost Customer Hours (LCH) stacked area plot. It also computes average EJT pre‑ and post‑upgrade for each line to assess the impact of investments.



## Data Sources

The analysis is based on data provided by Transport for London (TfL) and compiled in several CSV files.

- `station_annual_counts_updated.csv` – annual entry/exit counts for each station.
- `station_line_mapping.csv` – mapping of stations to the lines they serve.
- `line_annual_ridership_updated.csv` – line‑level ridership allocated from station counts.
- `station_ridership_period.csv` – ridership estimates by line and financial period.
- `lu_panel.csv` – network and line‑level performance metrics by period (Excess Journey Time, Lost Customer Hours, kilometres operated, percentage of schedule).
- `reliability_metrics_clean.csv` – metrics on rolling stock reliability, escalator/lift availability and service control failures.
- `lu_panel_with_station_ridership_upgrades.csv` – consolidated panel with network metrics, ridership, reliability measures and upgrade timeline (with `upgrade_year` and phase indicators).

These files are located in the `../data/raw` folder relative to this notebook.


In [ ]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configure seaborn style
sns.set_theme(style="whitegrid")
%matplotlib inline


In [ ]:

# Define base data directory relative to notebook
DATA_DIR = os.path.join('..', 'data', 'raw')

# Paths to the primary CSV files
panel_file = os.path.join(DATA_DIR, 'lu_panel_with_station_ridership_upgrades.csv')

# If you need other datasets later, define their paths here (not used in this analysis)
# station_counts_file = os.path.join(DATA_DIR, 'station_annual_counts_updated.csv')
# station_line_file = os.path.join(DATA_DIR, 'station_line_mapping.csv')


In [ ]:

# Load the consolidated panel with upgrade information

df = pd.read_csv(panel_file)

# Load the original line-level data to retrieve Lost Customer Hours (lch_line)
lu_panel_base = pd.read_csv(os.path.join(DATA_DIR, 'lu_panel.csv'))
base_lch = lu_panel_base[['financial_period', 'line', 'lch_line']]

# Drop existing lch_line in df (if present) and merge base_lch onto df
if 'lch_line' in df.columns:
    df = df.drop(columns=['lch_line'])
df = df.merge(base_lch, on=['financial_period', 'line'], how='left')

# Display basic information about the DataFrame
print('Data shape:', df.shape)
df.head()



### Column descriptions

The `lu_panel_with_station_ridership_upgrades` dataset contains the following columns:

- **financial_period**: Fiscal period in the format `pp_yy/yy` (e.g., `02_11/12`).
- **financial_year**: Fiscal year range.
- **period**: Period number (1–13) within the fiscal year.
- **days**: Number of days in the period.
- **period_end**: End date of the period (YYYY‑MM‑DD).
- **line**: Name of the Underground line (e.g., Central, Victoria).
- **lch_network**: Lost customer hours (LCH) across the network for the period.
- **ejt_network**: Excess journey time (EJT) across the network (in minutes).
- **operated_kms_network**: Kilometres operated across the network.
- **pct_scheduled**: Percentage of scheduled kilometres operated.
- **lch_line**: LCH allocated to the specific line.
- **ejt_line**: EJT allocated to the line.
- **operated_kms_line**: Kilometres operated on the line.
- **station_alloc_ridership**: Ridership (entries/exits) allocated to the line and period.
- **period_year**: Calendar year corresponding to the period.
- **upgrade_year**: Year of the major capacity upgrade for the line (if applicable).
- **train_upgrade_year**: Year of the rolling‑stock upgrade (sub‑surface lines).
- **signalling_upgrade_year**: Year of the signalling upgrade (sub‑surface lines).
- **post_upgrade**: Binary indicator (1 if period_year ≥ upgrade_year).
- **post_train_upgrade**: Binary indicator (1 if period_year ≥ train_upgrade_year).
- **post_signalling_upgrade**: Binary indicator (1 if period_year ≥ signalling_upgrade_year).


In [ ]:

# Ensure period is integer for time series calculation
# The dataset may contain strings or floats; convert to integer safely

df['period_int'] = df['period'].astype(int)
# Convert period_year to integer (handle strings like '2011/2012' by taking first four digits)
df['period_year_int'] = df['period_year'].astype(int)

# Create a unique time index for plotting (year * 100 + period)
df['time_id'] = df['period_year_int'] * 100 + df['period_int']

# Sort by time_id for consistent plotting
df = df.sort_values('time_id')

# Display the new columns
print(df[['period_year_int','period_int','time_id']].head())


In [ ]:

# Pivot the data to wide format for EJT trend

ejt_trend = df.pivot_table(index='time_id', columns='line', values='ejt_line', aggfunc='mean')

# Plot EJT trend for each line
plt.figure(figsize=(12, 6))
for line in ejt_trend.columns:
    plt.plot(ejt_trend.index, ejt_trend[line], marker='o', label=line)
plt.xlabel('Time (year * 100 + period)')
plt.ylabel('Excess Journey Time (minutes)')
plt.title('Excess Journey Time Trend by Line')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:

# Create a heatmap of EJT by line and time_id

ejt_heatmap_data = df.pivot_table(index='line', columns='time_id', values='ejt_line', aggfunc='mean')

plt.figure(figsize=(14, 6))
sns.heatmap(ejt_heatmap_data, cmap='YlOrRd', cbar_kws={'label': 'Excess Journey Time (minutes)'}, linewidths=0.2)
plt.title('Excess Journey Time Heatmap by Line and Period')
plt.xlabel('Time (year * 100 + period)')
plt.ylabel('Line')
plt.tight_layout()
plt.show()


In [ ]:

# Pivot data for stacked area plot of Lost Customer Hours

lch_stack_data = df.pivot_table(index='time_id', columns='line', values='lch_line', aggfunc='sum')

# Normalize data if there are missing years/periods by forward filling
lch_stack_data = lch_stack_data.fillna(0)

# Plot stacked area
plt.figure(figsize=(12, 6))
lch_stack_data.sort_index().plot(kind='area', stacked=True, figsize=(12, 6), colormap='tab10')
plt.xlabel('Time (year * 100 + period)')
plt.ylabel('Lost Customer Hours')
plt.title('Lost Customer Hours Stacked Area Plot by Line')
plt.legend(title='Line', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:

# Compute average EJT per line before and after major upgrade

ejt_upgrade_stats = df.groupby(['line', 'post_upgrade'])['ejt_line'].mean().reset_index()

# Pivot for easier viewing
pre_post_ejt = ejt_upgrade_stats.pivot(index='line', columns='post_upgrade', values='ejt_line')
pre_post_ejt.columns = ['Pre-upgrade', 'Post-upgrade']
pre_post_ejt = pre_post_ejt.sort_index()

print('Average EJT by line (minutes):')
pre_post_ejt



### Interpretation of pre vs. post upgrade EJT

The table above shows the mean excess journey time (in minutes) for each Underground line before and after its major capacity upgrade. A reduction from the pre‑upgrade to the post‑upgrade column suggests that the upgrade may have improved service performance. Lines without upgrades during 2010–2024 are labelled as missing (`NaN`) in the `upgrade_year` column, and thus all periods are treated as pre‑upgrade.
